In [28]:
import pandas as pd
import datetime as datetime

In [2]:
df = pd.read_csv("FinalFinal.csv")

In [68]:
df.columns
# select non join columns and ML needed columns
columns = ['MNL_FEAT_1', 'MNL_MATE_1', 'MNL_LENGTH', 'MNL_INSTAL',
       'Width', 'NEAR_DIST', 'MUSYM', 'ARTCLASS', 'BLOCKNBR', 
       'SPEEDLIMIT', 'SURFACEWID', 'SURFACETYP', 'SLOPE_PCT', 'Size', 'DATE']
non_joins = df[columns]
# non_joins.head()

In [69]:
# select only years
non_joins['MNL_INSTAL'] = pd.to_datetime(non_joins['MNL_INSTAL'], format='%m/%d/%Y %H:%M:%S')
non_joins['MNL_INSTAL'] = non_joins['MNL_INSTAL'].map(lambda x: x.year)

non_joins['DATE'] = pd.to_datetime(non_joins['DATE'], format='%m/%d/%Y %H:%M:%S')
non_joins['DATE'] = non_joins['DATE'].map(lambda x: x.year)

#non_joins.head()

In [70]:
# filter out NaN from DATE col
breaks_df = non_joins[non_joins['DATE'].notna()]
# set size column to width column and drop size 
breaks_df['Width'] = breaks_df['Size']
breaks_df = breaks_df.drop('Size', axis=1)
breaks_df.head()

,MNL_FEAT_1,MNL_MATE_1,MNL_LENGTH,MNL_INSTAL,Width,NEAR_DIST,MUSYM,ARTCLASS,BLOCKNBR,SPEEDLIMIT,SURFACEWID,SURFACETYP,SLOPE_PCT,DATE
43,Mainline,Concrete,40.54,1957,6,699.766876,3056,0,1100,20,20,AC,2,2014.0
44,Mainline,Concrete,40.54,1957,6,699.766876,3056,0,1100,20,20,AC,2,2014.0
81,Mainline,Reinforced Concrete Pipe,25.65,1973,8,2296.768775,3055,0,100,20,38,ST,4,2017.0
82,Mainline,Reinforced Concrete Pipe,25.65,1973,8,2296.768775,3055,0,100,20,38,ST,4,2017.0
120,Mainline,Concrete,182.67,1964,4,221.097406,3056,0,3400,20,25,PCC,12,2015.0


In [ ]:
# for loop - has start year and end year
# ex: first yr=2009 end yr=2012
# grab everything from 3yr window 